In [ ]:
"2024-01-02-Dill Picking Error with QApplication.ipynb"


In [ ]:
# Problem: QApplication cannot be pickled error occurs when trying to pickle pipeline after adding RankOrder global computation results despite not ever being used.

# Solution: Unsolved

In [ ]:
# Observation: Sometimes specific result can be successfully picked while entire pipeline cannot:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import save_rank_order_results, SaveStringGenerator
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData

save_rank_order_results(curr_active_pipeline, day_date=f"{DAY_DATE_TO_USE}_322pm") # "2024-01-02_301pm" "2024-01-02_322pm" 322pm

In [ ]:
# 2023-01-02 - Fails SOMETIMES, even when called immediately after the above save command which worked:
curr_active_pipeline.save_global_computation_results()

# Clearing and reloading the notebook, skipping these computations, and then re-applying them to the pipeline does allow resaving with `curr_active_pipeline.save_global_computation_results()` for some reason.



In [ ]:
curr_active_pipeline.global_computation_results.computed_data['RankOrder']

import dill as pickle

def diagnose_pickling_issues(object_to_pickle):
   """Intellegently diagnoses which property on an object is causing pickling via Dill to fail."""

   try:
       # Attempt to pickle the object directly
       pickle.dumps(object_to_pickle)
   except pickle.PicklingError as e:
       # If pickling fails, initiate a diagnostic process
       print(f"Pickling error encountered: {e}")

       # Gather information about the object's attributes
       object_attributes = [attr for attr in dir(object_to_pickle) if not attr.startswith("__")]

       # Isolate problematic attributes through iterative testing
       problematic_attribute = None
       for attribute in object_attributes:
           try:
               pickle.dumps(getattr(object_to_pickle, attribute))
           except pickle.PicklingError:
               problematic_attribute = attribute
               break

       # Provide informative output
       if problematic_attribute:
           print(f"Identified problematic attribute: {problematic_attribute}")
           print("Potential causes:")
           print("- Attribute contains unpicklable data types (e.g., lambda functions, file objects).")
           print("- Attribute refers to external resources (e.g., database connections).")
           print("- Attribute has circular references within the object's structure.")
       else:
           print("Unable to isolate the specific attribute causing the pickling error.")
           print("Consider:")
           print("- Examining the object's structure and dependencies for potential conflicts.")
           print("- Providing a minimal reproducible example for further analysis.")

   else:
       # If pickling succeeds, indicate no issues found
       print("No pickling issues detected.")


diagnose_pickling_issues(curr_active_pipeline.global_computation_results.computed_data['RankOrder'])

# make a copy of an object

In [ ]:
rank_order_output_path = Path(r'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\2023-12-22_807pm-minimum_inclusion_fr-5-included_qclu_values-[1, 2]RankOrder.pkl').resolve()


from pyphocorehelpers.Filesystem.pickling_helpers import custom_dump, custom_dumps
import dill.detect
dill.detect.trace(True)

# dill.detect.badobjects(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__)

test_obj = curr_active_pipeline.global_computation_results.computed_data['RankOrder'] # pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations.RankOrderComputationsContainer
dill.detect.badtypes(test_obj)

# dill.detect.errors(test_obj)

# with dill.detect.trace(True):
# saveData(rank_order_output_path, (curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__,))
# custom_dumps(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__)
#  with open(pkl_path, file_mode) as dbfile: 
# 	# source, destination
# 	# pickle.dump(db, dbfile)
# 	custom_dump(db, dbfile) # ModuleExcludesPickler
# 	dbfile.close()
	
# # dumps(squared)
# custom_dump(db, dbfile) # ModuleExcludesPickler
# saveData(rank_order_output_path, (curr_active_pipeline.global_computation_results.computed_data['RankOrder'].__dict__,))
